In [31]:
import os

def load_environment_variables(file_path):
    with open(file_path, 'r') as file:
        for line in file:
            key, value = line.strip().split('=', 1)
            os.environ[key] = value

if __name__ == "__main__":
    load_environment_variables('env_vars.txt')
    # Teste para verificar se as variáveis foram carregadas
    print(f"PASSWORD: {os.environ.get('DATABASE')}")


PASSWORD: ETL


In [32]:
import pandas as pd
import time
from datetime import datetime, timedelta
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support.ui import Select
import os
import xmltodict
import pandas as pd
from tqdm import tqdm
import zipfile
import io
import numpy as np
import requests
from datetime import datetime
import json
import pandas as pd
import psycopg2
from urllib.parse import quote
import sqlalchemy
from sqlalchemy import create_engine #importing sqlalchemy engine to create engine for the database
from sqlalchemy import inspect


host= os.getenv('HOST')
database=os.getenv('DATABASE')
port= os.getenv('PORT')
user=os.getenv('USER')
password=os.getenv('PASSWORD')
email=os.getenv('EMAIL')
password_sige=os.getenv('SIGE_PASSWORD')


engine = create_engine(f'postgresql://{user}:%s@{host}:{port}/{database}' %quote(password), echo=False)

def conecta_db():
  con = psycopg2.connect(host=host, 
                         database=database,
                         port=port,
                         user=user, 
                         password=password)
  
  return con
  


#Consulta no SQL
def consultar_db(sql):
  con = conecta_db()
  cur = con.cursor()
  cur.execute(sql)
  recset = cur.fetchall()
  colunas = [tup[0] for tup in cur.description]
  registros = []
  for rec in recset:
    registros.append(rec)
  con.close()
  df = pd.DataFrame(registros)
  df.columns = colunas
  return df



In [35]:




def esperar_input_ter_valor(driver, xpath, valor_esperado, timeout=240):
    """Espera até que o input tenha o valor desejado."""
    end_time = time.time() + timeout
    while time.time() < end_time:
        try:
            elemento = WebDriverWait(driver, 10).until(
                EC.visibility_of_element_located((By.XPATH, xpath))
            )
            elemento.clear()
            elemento.send_keys(valor_esperado)
            time.sleep(10)  # Pequena pausa para garantir que o valor seja atualizado
            if elemento.get_attribute('value') == valor_esperado:
                return True
        except Exception as e:
            print(f"Erro ao definir o valor do input: {e}")
            time.sleep(1)  # Espera antes de tentar novamente
    return False

def fechar_popup_se_presente(driver, xpath):
    """Fecha o popup se estiver presente."""
    try:
        popup_elements = driver.find_elements(By.XPATH, xpath)
        for popup_element in popup_elements:
            if popup_element.is_displayed():
                popup_element.click()
                break
    except Exception as e:
        print(f"Erro ao fechar o popup: {e}")

def realizar_login():
    service = Service(ChromeDriverManager().install())
    download_folder = "XML_SIGE"

    # Configurações do navegador
    chrome_options = Options()
    prefs = {
        "download.default_directory": download_folder,  # Define o diretório padrão de downloads
        "download.prompt_for_download": False,          # Desativa a caixa de diálogo de download
        "download.directory_upgrade": True,             # Ativa a gestão de múltiplos downloads
        #"safebrowsing.enabled": True                    # Ativa a proteção de navegação
    }
    chrome_options.add_experimental_option("prefs", prefs)
    nav = webdriver.Chrome(service=service)
    nav.get('https://app.sigecloud.com.br/login')
    nav.find_element(By.XPATH, '/html/body/form/div[3]/div/div[2]/div[1]/input').send_keys(email)
    nav.find_element(By.XPATH,'/html/body/form/div[3]/div/div[2]/div[2]/input').send_keys(password_sige)
    nav.find_element(By.XPATH, '/html/body/form/div[3]/div/div[2]/input').click()
    return nav

def selecionar_intervalo_datas_e_download(nav, data_inicio):
    nav.get('https://app.sigecloud.com.br/NFe/DownloadXML?tipodoc=XMLNFe')

    # Espera até que o select esteja presente
    select_element = WebDriverWait(nav, 10).until(
        EC.presence_of_element_located((By.XPATH, '/html/body/form/div[4]/div[2]/div/div[3]/div[1]/div[1]/ul/li[3]/select'))
    )
    
    # Cria uma instância Select e seleciona a opção "XML NFCe" pelo texto visível
    select = Select(select_element)
    select.select_by_visible_text("XML NFCe")

    # Verificação opcional para assegurar que a opção correta está selecionada
    selected_option = select.first_selected_option
    if selected_option.text != "XML NFCe":
        print("A opção 'XML NFCe' não está selecionada.")
        return None
    else:
        print("A opção 'XML NFCe' está corretamente selecionada.")

    data_fim = data_inicio + timedelta(days=14)
    data_inicio_str = data_inicio.strftime("%d/%m/%Y")
    data_fim_str = data_fim.strftime("%d/%m/%Y")

    input_data_inicio_xpath = '/html/body/form/div[4]/div[2]/div/div[3]/div[1]/div[1]/ul/li[5]/input'
    input_data_fim_xpath = '/html/body/form/div[4]/div[2]/div/div[3]/div[1]/div[1]/ul/li[6]/input'
    
    if esperar_input_ter_valor(nav, input_data_inicio_xpath, data_inicio_str) and esperar_input_ter_valor(nav, input_data_fim_xpath, data_fim_str):
        nav.find_element(By.XPATH, '/html/body/form/div[4]/div[2]/div/div[3]/div[1]/div[1]/ul/li[7]/button').click()
        # Implemente a lógica específica de espera pela conclusão do download aqui
        print(f'Data {data_inicio_str} a {data_fim_str} processada com sucesso.')
        time.sleep(10)
        return [data_inicio_str, data_fim_str]
    else:
        print("Não foi possível definir as datas nos inputs corretamente.")
        return None

def salvar_datas_csv(datas, filename='datas_processadas.csv'):
    df = pd.DataFrame(datas, columns=['Data Inicio', 'Data Fim'])
    df.to_csv(filename, mode='a', header=not os.path.exists(filename), index=False)

if __name__ == "__main__":
    try:
        nav = realizar_login()
        datas_processadas = []
        ultima_data_tabela = consultar_db(''' SELECT MAX("dhEmi") AS max_dhemi FROM sige_notafiscal_xml ''')
        data_inicio = ultima_data_tabela.at[0, 'max_dhemi'] - timedelta(days=90)
        

        while data_inicio < datetime.now():
            data_fim = data_inicio + timedelta(days=14)
            if data_fim > datetime.now():
                data_fim = datetime.now()

            resultado = selecionar_intervalo_datas_e_download(nav, data_inicio)
            if resultado:
                datas_processadas.append(resultado)
                data_inicio += timedelta(days=15)
            else:
                print(f"Falha ao processar o intervalo iniciado em {data_inicio.strftime('%d/%m/%Y')}.")
                break

        salvar_datas_csv(datas_processadas)
        print("Processo concluído com sucesso!")
    except Exception as e:
        print(f"Ocorreu um erro: {e}")


A opção 'XML NFCe' está corretamente selecionada.
Erro ao definir o valor do input: Message: javascript error: {"status":12,"value":"Element is not currently interactable and may not be manipulated"}
  (Session info: chrome=124.0.6367.63)
Stacktrace:
	GetHandleVerifier [0x0078C113+48259]
	(No symbol) [0x0071CA41]
	(No symbol) [0x00610A17]
	(No symbol) [0x00615B58]
	(No symbol) [0x00617652]
	(No symbol) [0x006176D0]
	(No symbol) [0x0064935C]
	(No symbol) [0x00670D9C]
	(No symbol) [0x00646E25]
	(No symbol) [0x00671034]
	(No symbol) [0x00689B9C]
	(No symbol) [0x00670B36]
	(No symbol) [0x0064570D]
	(No symbol) [0x006462CD]
	GetHandleVerifier [0x00A46533+2908323]
	GetHandleVerifier [0x00A83B4B+3159739]
	GetHandleVerifier [0x0082505B+674763]
	GetHandleVerifier [0x0082B21C+699788]
	(No symbol) [0x00726244]
	(No symbol) [0x00722298]
	(No symbol) [0x0072242C]
	(No symbol) [0x00714BB0]
	BaseThreadInitThunk [0x76317BA9+25]
	RtlInitializeExceptionChain [0x7770BE3B+107]
	RtlClearBits [0x7770BDBF+19

KeyboardInterrupt: 